In [1]:
import cv2

In [2]:
cv2.__version__

'4.0.1'

In [3]:
cv2.TrackerBoosting_create()

<TrackerBoosting 000001E83B62B530>

In [4]:
import cv2
import numpy as np
import math
import typing
import os
def show(frame):
    cv2.imshow("test", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
def plot_point(frame,x,y,color=(0,255,0)):
    radius = 1
    thickness = -1
    return cv2.circle(frame, (x,y), radius, color, thickness)
def plot_line(frame,p1,p2,color=(0,191,255)):
    thickness = 2
    return cv2.line(frame, (p1[0],p1[1]), (p2[0],p2[1]), color, thickness)
def getIdealPointGroup(points):
  pointGroups = []
  index = 0
  subgroup = [points[0]]

  for i in range(len(points) - 1):
    prevPoint = points[i]
    currentPoint = points[i+1]

    if (abs(int(prevPoint[0])-int(currentPoint[0])) <= 1) and (abs(int(prevPoint[1])-int(currentPoint[1])) <= 1):
      subgroup.append(currentPoint)
    else:
      pointGroups.append(subgroup[:])
      subgroup = [currentPoint]

  pointGroups.append(subgroup)

  mainPointGroup = []
  maxPointGroupSize = 0

  for group in pointGroups:
    if len(group) > maxPointGroupSize:
      maxPointGroup = group
      maxPointGroupSize = len(group)

  return maxPointGroup

# Gets all the contours for certain image
  

def get_bottom_left(points,img,index = None):
  if not index is None:
    i = int(index*len(points))
    return points[i], index 
  max_y = len(img[0])
  index = 0
  dist = 100000
  count = 0
  for i in points:
    if (i[0])**2+(max_y-i[1])**2<dist:
      index = count
      dist = (i[0])**2+(max_y-i[1])**2
    count+=1
  return points[index],index/len(points)

def loadvideo(filename: str) -> np.ndarray:
    """Loads a video from a file.

    Args:
        filename (str): filename of video

    Returns:
        A np.ndarray with dimensions (channels=3, frames, height, width). The
        values will be uint8's ranging from 0 to 255.

    Raises:
        FileNotFoundError: Could not find `filename`
        ValueError: An error occurred while reading the video
    """

    if not os.path.exists(filename):
        raise FileNotFoundError(filename)
    capture = cv2.VideoCapture(filename)

    frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

    v = np.zeros((frame_count, frame_height, frame_width, 3), np.uint8)

    for count in range(frame_count):
        ret, frame = capture.read()
        if not ret:
            raise ValueError("Failed to load frame #{} of {}.".format(count, filename))

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        v[count] = frame

    v = v.transpose((3, 0, 1, 2))

    return v
def savevideo(filename: str, array: np.ndarray, fps: typing.Union[float, int] = 1):
    """Saves a video to a file.

    Args:
        filename (str): filename of video
        array (np.ndarray): video of uint8's with shape (channels=3, frames, height, width)
        fps (float or int): frames per second

    Returns:
        None
    """

    c, f, height, width = array.shape

    if c != 3:
        raise ValueError("savevideo expects array of shape (channels=3, frames, height, width), got shape ({})".format(", ".join(map(str, array.shape))))
    fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
    out = cv2.VideoWriter(filename, fourcc, fps, (width, height))

    for i in range(f):
        out.write(array[:, i, :, :].transpose((1, 2, 0)))

        
def obtainContourPoints(img):
  # read image

  rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  # set lower and upper bounds on blue color
  lower = (200,0,0)
  upper = (255,200,200)

  # threshold and invert so hexagon is white on black background
  thresh = cv2.inRange(rgb, lower, upper)
  # print(thresh.shape,np.min(thresh),np.max(thresh),type(thresh))
  # show(thresh)
  # get contours
  result = np.zeros_like(thresh)
  im2, contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  #print(im2,contours)
  #cv2.drawContours(img, im2, -1, (0,255,0), 3)

  im2 = np.array(im2)
  index_1 = 0
  index_2 = 0
  for i in im2:
    if i.shape[0]>im2[index_1].shape[0]:
      index_1=index_2
    index_2+=1
  rec = cv2.minAreaRect(im2[index_1])
  color = (0, 0, 255) 
  thickness = 2
  box = cv2.boxPoints(rec)
  box = np.int0(box)
  indexes = [0,1]
  for i in range(0,len(box)):
    for k in range(0,len(indexes)):
      if box[i][1]>box[indexes[k]][1] and not i in indexes:
        indexes[k]=i

  #cv2.drawContours(img,[box],0,(191,0,255),2)
  #img = plot_line(img,box[indexes[0]],box[indexes[1]])
  #img = plot_point(img,box[indexes[0],0],box[indexes[0],1])
  #img = plot_point(img,box[indexes[1],0],box[indexes[1],1])
  #cv2.rectangle(img,rec,color,thickness)
  return img, [box[indexes[0]].tolist(),box[indexes[1]].tolist()]

In [5]:
import cv2
import numpy as np
# https://gist.github.com/epiception/ac8195435976f6d2356869589b7157de
'''KALMAN FILTERING CLASS FOR N 2D POINTS'''
'''Kalman filtering for selected points in an image using OpenCV cv2.kalmanFilter class in Python '''
class Kalman_Filtering:

    def __init__(self,n_points):
        self.n_points = n_points

    def initialize(self):

        n_states = self.n_points * 4
        n_measures = self.n_points * 2
        self.kalman = cv2.KalmanFilter(n_states,n_measures)
        # print(self.kalman)
        kalman = self.kalman
        kalman.transitionMatrix = np.eye(n_states, dtype = np.float32)
        #kalman.processNoiseCov = np.eye(n_states, dtype = np.float32)*0.9
        kalman.measurementNoiseCov = np.eye(n_measures, dtype = np.float32)*0.0005

        kalman.measurementMatrix = np.zeros((n_measures,n_states), np.float32)
        dt = 1

        self.Measurement_array = []
        self.dt_array = []

        for i in range(0,n_states,4):
            self.Measurement_array.append(i)
            self.Measurement_array.append(i+1)

        for i in range(0,n_states):
            if i not in self.Measurement_array:
                self.dt_array.append(i)

        # print(self.dt_array)
        # print(self.Measurement_array)
        #Transition Matrix for [x,y,x',y'] for n such points
        # format of first row [1 0 dt 0 .....]
        for i, j in zip(self.Measurement_array, self.dt_array):
            kalman.transitionMatrix[i,j] = dt;

        #Measurement Matrix for [x,y,x',y'] for n such points
        # format of first row [1 0 0 0 .....]
        for i in range(0,n_measures):
            kalman.measurementMatrix[i,self.Measurement_array[i]] = 1


        #print('TRANSITION Matrix:')
        #print(kalman.transitionMatrix)

        #print('MEASUREMENT Matrix:')
        #print(kalman.measurementMatrix)



    def predict(self,points):

        pred = []
        input_points = np.float32(np.ndarray.flatten(points))
        #Correction Step
        # print(input_points)
        self.kalman.correct(input_points)
        #Prediction step
        tp = self.kalman.predict()

        for i in self.Measurement_array:
            pred.append(int(tp[i]))

        return pred
def show(frame):
    cv2.imshow("test", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
def load(vid):
    vid = loadvideo(vid)
    #vid = np.transpose(vid,(1,2,3,0))
    vid = vid[:,:,:,:112]
    return vid

In [6]:
def midpoint(point1,point2):
    return (point1+point2)/2
def change(points):
    point_arr = [points[0]]
    for i in points[1:]:
        point_arr.append(midpoint(point_arr[-1],i))
    return np.array(point_arr)
import matplotlib.pyplot as plt


In [7]:
key_points = []
import pandas as pd
df = pd.read_csv('C:\\Users\\TheurerJ\\Documents\\John\\dynamic\\dynamic\\LeftX_RightY.csv')
from tqdm import tqdm
folder = 'C:\\Users\\TheurerJ\\Documents\\Data\\segmented'
output = 'C:\\Users\\TheurerJ\\Documents\\Data\\midpoint'
filenames = []
frame_num = []
x1 = []
y1 = []
x2 = []
y2 = []
key_frames = []
delta_x1 = []
delta_y1 = []
delta_x2 = []
delta_y2 = []
for vid in tqdm(os.listdir(folder)):
    try:
        if len(df[df.FileName==vid])>0:
            temp = df[df.FileName==vid]
            exists = True

        video_file = os.path.join(folder,vid)
        first = load(video_file)
        spare = first.copy()
        first = np.transpose(first,(1,2,3,0))
        video = []
        guess = None
        vertexes = []
        kf = Kalman_Filtering(len(first))
        kf.initialize()
        for frame in range(0,len(first)):
            #print(temp.Frame==frame,frame)
            img,points = obtainContourPoints(first[frame])
            if np.linalg.norm(points[0])>np.linalg.norm(points[1]):
                c = points[0].copy()
                points[0] = points[1].copy()
                points[1] = c
                print("this happens")
            if exists and len(temp[temp.Frame ==frame]) == 1:
                
                delta_x1.append(points[0][0]-temp[temp.Frame ==frame].X1.tolist()[0])
                delta_y1.append(points[0][1]-temp[temp.Frame ==frame].Y1.tolist()[0])
                delta_x2.append(points[1][0]-temp[temp.Frame ==frame].X2.tolist()[0])
                delta_y2.append(points[1][1]-temp[temp.Frame ==frame].Y2.tolist()[0])
            vertexes.append(points)
            
            # video.append(img)
            #show(img)

        #for point in vertexes:
        #change(np.array(vertexes))
        ok = np.array(vertexes)
        # print(ok.shape)
        first_points = change(ok[:,0,:])
        second_points = change(ok[:,1,:])
        #print(second_points)
        # print(kf.predict(ok[:,0,:]))
        #delta(ok[:,0,:])
        for i in range(0,len(first)):
            filenames.append(vid)
            key_frames.append(i)
            x1.append(int(ok[i,0,0]))
            y1.append(int(ok[i,0,1]))
            x2.append(int(ok[i,1,0]))
            y2.append(int(ok[i,1,1]))
            #x1.append(int(first_points[i,0]))
            #y1.append(int(first_points[i,1]))
            #x2.append(int(second_points[i,0]))
            #y2.append(int(second_points[i,1]))
            img = plot_point(first[i],int(first_points[i,0]),int(first_points[i,1]))
            
            img = plot_point(img,int(second_points[i,0]),int(second_points[i,1]))
            img = plot_point(img,ok[i,0,0],ok[i,0,1],color=(255,0,255))
            img = plot_point(img,ok[i,1,0],ok[i,1,1],color=(255,0,255))
            # print(ok[i,0,0],ok[i,0,1],'-',int(first_points[i,0]),int(first_points[i,1]))
            # print(ok[i,1,0],ok[i,1,1],'-',int(second_points[i,0]),int(second_points[i,1]))
            #show(img)
            video.append(img)
        video = np.transpose(np.array(video),(3,0,1,2))

        key_points.append(np.array(vertexes))
        savevideo(os.path.join("Midpoint2",vid),video,fps=30)
    except:
        print(vid)


  0%|                                                                                 | 1/3229 [00:00<12:54,  4.17it/s]

this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens


  0%|                                                                                 | 4/3229 [00:00<11:10,  4.81it/s]

this happens
this happens
this happens
this happens
this happens


  0%|▏                                                                                | 5/3229 [00:01<12:23,  4.33it/s]

...

this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens


100%|█████████████████████████████████████████████████████████████████████████████▉| 3228/3229 [11:00<00:00,  4.89it/s]

this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens
this happens


100%|██████████████████████████████████████████████████████████████████████████████| 3229/3229 [11:00<00:00,  4.89it/s]

this happens


In [8]:
import pandas as pd
df = pd.DataFrame({"FileNames":filenames,"Frame":key_frames,"X1":x1,"Y1":y1,"X2":x2,"Y2":y2})

In [9]:
df.to_csv("Unsmoothed.csv")

In [10]:
hold_up = ok[:,0,:]
import math
def calc_degrees(point):
    return math.atan2(point[1], point[0])
for i in hold_up:
    print(calc_degrees(i))

1.0324218134486665
1.0444039625684765
1.0444039625684765
1.0559115503906717
1.0559115503906717
1.0444039625684765
1.0324218134486665
1.0366137518895122
1.0324218134486665
1.0366137518895122
1.0366137518895122
1.0303768265243125
1.0303768265243125
1.0240074859056494
1.0108565296805083
1.0108565296805083
1.0108565296805083
1.0108565296805083
0.9897140279975696
0.9897140279975696
0.9897140279975696
0.9897140279975696
0.982793723247329
0.982793723247329
0.9897140279975696
0.9897140279975696
0.9964914966201949
0.9861950706745702
1.0191413442663497
0.982793723247329
0.9964914966201949
0.9964914966201949
1.0108565296805083
1.0108565296805083
1.0108565296805083
1.0108565296805083
1.0108565296805083
1.0108565296805083
1.0108565296805083
1.017502001472604
1.017502001472604
0.9964914966201949
1.0108565296805083
1.0108565296805083
1.0108565296805083
1.003129902423017
1.003129902423017
1.003129902423017
1.0240074859056494
1.0096329145065444
1.0222469243443686
0.9987668055492562
1.004941999686837
1.

In [11]:
results = pd.DataFrame({"FileNames":filenames,'X1':delta_x1,'Y1':delta_y1,'X2':delta_x2,'Y2':delta_y2})
results['p1']=np.sqrt(results.X1**2+results.Y1**2)
results['p2']=np.sqrt(results.X2**2+results.Y2**2)

ValueError: arrays must all be same length

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(results['p1'],results['p2'])

In [ ]:
plt.scatter(results['X1'],results['X2'])

In [ ]:
plt.scatter(results['X1'],results['Y1'])

In [ ]:
print(np.mean(results['X2']))

In [ ]:
print(np.mean(results['Y2']))

In [ ]:
[[74 53]
 [74 53]
 [74 53]
 [74 53]
 [74 53]
 [74 51]
 [52 55]
 [72 55]
 [72 55]
 [72 55]
 [70 55]
 [70 55]
 [70 55]
 [70 55]
 [70 55]
 [70 55]
 [69 53]
 [69 53]
 [71 53]
 [71 53]
 [73 51]
 [73 51]
 [73 51]
 [73 51]
 [75 51]
 [75 51]
 [75 51]
 [75 51]
 [75 53]
 [75 53]
 [75 53]
 [75 53]
 [77 54]
 [77 54]
 [56 57]
 [76 55]
 [76 55]
 [76 55]
 [76 55]
 [76 55]
 [78 54]
 [78 54]
 [78 54]
 [79 52]
 [79 52]
 [77 54]
 [77 54]
 [74 54]
 [71 55]
 [71 55]
 [71 53]
 [71 53]
 [70 55]
 [70 55]
 [70 57]
 [70 57]
 [68 57]
 [68 57]
 [70 54]
 [70 54]
 [71 51]
 [71 51]
 [73 51]
 [73 51]
 [73 51]
 [74 53]
 [74 53]
 [74 53]
 [74 53]
 [56 55]
 [75 53]
 [75 53]
 [75 53]
 [75 53]
 [75 53]
 [78 55]
 [78 55]
 [78 55]
 [78 55]
 [77 53]
 [75 54]
 [73 55]
 [73 55]
 [71 55]
 [71 55]
 [71 55]
 [69 55]
 [69 57]
 [69 57]
 [69 57]
 [69 57]
 [69 57]
 [69 53]
 [69 53]
 [69 53]
 [71 51]
 [71 51]
 [71 51]
 [71 51]
 [72 53]
 [72 53]
 [74 53]
 [74 53]
 [76 55]
 [74 55]
 [74 55]
 [74 53]
 [72 55]
 [70 55]
 [70 55]
 [68 55]
 [68 55]
 [68 55]
 [66 56]
 [66 56]
 [66 56]
 [65 54]
 [65 54]
 [65 54]
 [67 53]
 [69 53]
 [69 53]
 [69 53]
 [69 53]
 [52 55]
 [52 55]
 [52 55]
 [71 54]
 [71 56]
 [71 56]
 [71 56]
 [71 56]
 [74 55]
 [74 55]
 [74 55]
 [74 55]
 [74 55]
 [74 55]
 [74 55]
 [76 54]
 [76 54]
 [76 54]
 [76 54]
 [74 55]
 [74 55]
 [74 55]
 [71 54]
 [69 55]
 [69 55]
 [69 57]
 [69 57]
 [69 57]
 [68 55]
 [68 55]
 [68 55]
 [68 53]
 [68 53]
 [70 54]
 [71 50]
 [73 50]
 [73 50]
 [73 50]
 [73 52]
 [74 54]
 [74 54]
 [74 54]
 [74 54]
 [74 54]
 [74 54]
 [76 53]
 [76 53]]